In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.preprocessing import StandardScaler as SS # z-score standardization 
from sklearn.cluster import KMeans, DBSCAN # clustering algorithms
from sklearn.metrics import silhouette_score # used as a metric to evaluate the cohesion in a cluster
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("Spotify_Youtube.csv", index_col = 0)

In [3]:
#SPOTIPY_CLIENT_ID = "e67f8aadfcd2400b912e1c372e332ec4"
#SPOTIPY_CLIENT_SECRET = "6749f19d38584e3fa8b955253f4d5444"
#SPOTIPY_REDIRECT_URI = "http://localhost:3000"
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests

CLIENT_ID = "e67f8aadfcd2400b912e1c372e332ec4"
CLIENT_SECRET = "6749f19d38584e3fa8b955253f4d5444"

AUTH_URL = "https://accounts.spotify.com/api/token"
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET, })

#Convert response to JSON
auth_response_data = auth_response.json()

#Save the access token
access_token = auth_response_data['access_token']

#Need to pass access token into header to send properly formed GET request to API server
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [4]:
def getSimilarity(obj1, obj2):
  len1 = len(obj1)
  len2 = len(obj2)
  if not (len1 == len2):
    print ("Error: Compared objects must have same number of features.")
    sys.exit()
    return 0
  else:
    similarity = []
    similarity = obj1-obj2
    similarity = np.sum((similarity**2.0) / 10.0)
    similarity = 1 - math.sqrt(similarity)
    return similarity

In [7]:
def recommendation(url):
    q = url.split('?')
    p = q[0].split('/')
    
    BASE_URL = 'https://api.spotify.com/v1/audio-features/'
    FINAL_URL = BASE_URL + p[4]
    r = requests.get(FINAL_URL, headers=headers)
    r = r.json()

    x = pd.DataFrame.from_dict(r, orient = 'index')
    x = x.transpose()
    y = x[['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 
       'instrumentalness', 'liveness', 'valence', 'tempo']]
    y = y.rename(columns={'danceability': 'Danceability', 'energy': 'Energy', 'key': 'Key', 'loudness': 'Loudness', 
                      'speechiness': 'Speechiness', 'acousticness': 'Acousticness', 'instrumentalness': 'Instrumentalness', 
                     'liveness': 'Liveness', 'valence': 'Valence', 'tempo': 'Tempo'})

    df = data.drop(columns=["Artist", "Url_spotify", "Track", "Album", "Album_type", "Uri", "Duration_ms", "Url_youtube", "Title", "Channel", "Views", "Likes", "Comments", "Description", "Licensed", "official_video", "Stream"],inplace=False)
    df1 = df.dropna()

    df2 = pd.concat([df1, y])

    df2.loc[len(df.index)] = df2.mean()
    norm_arr = pd.DataFrame()
    for col in df2:
        norm_arr[col] = df2[col] /df2[col].abs().max()
    
    new_col = []
    for index, row in norm_arr.iterrows():
        score = getSimilarity(np.array(row), norm_arr.iloc[-1])
        new_col.append(score)
    
    
    norm_arr["sim_score"] = new_col
    norm_arr = norm_arr[:-1]

    norm_sliced = norm_arr[["sim_score"]]

    kmeans1 = KMeans(n_clusters=5, random_state=42).fit(norm_sliced)
    clusters = kmeans1.predict(norm_sliced)
    norm_sliced["clusters"] = clusters

    c = norm_sliced.iloc[-1]["clusters"]

    m = norm_sliced[norm_sliced["clusters"] == c].index
    m2 = m.values
    g = np.random.choice(m2, size = 5, replace = False)
    
    print("Five recommended songs for you are: ")
    for i in g:
        print(data.iloc[i]['Track'] + " by " + data.iloc[i]["Artist"])

In [8]:
recommendation("https://open.spotify.com/track/2zYzyRzz6pRmhPzyfMEC8s?si=b6731663be404b49")

Five recommended songs for you are: 
I'll Be There by Jess Glynne
Lovely Day by Bill Withers
Waiting On the World to Change by John Mayer
Arthur's Theme (Best That You Can Do) by Christopher Cross
Soy El Ratón by Código FN
